# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import FeatureUnion
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
# load data from database
engine = create_engine('sqlite:///Disaster_Response2.db')
df = pd.read_sql_table('Disaster_Response_Table2',engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normlized = [normlizer.stem(word) for word in tokens if word not in stop_words]
    
    return normlized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

np.random.seed(13)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_prediction_train = pipeline.predict(X_train)
y_prediction_test = pipeline.predict(X_test)


In [12]:
# Print classification report on test data
print(classification_report(Y_test.values,y_prediction_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.91      0.88      4967
               request       0.79      0.44      0.56      1120
                 offer       0.00      0.00      0.00        23
           aid_related       0.72      0.60      0.66      2679
          medical_help       0.48      0.06      0.11       519
      medical_products       0.67      0.11      0.19       321
     search_and_rescue       0.56      0.06      0.11       153
              security       0.00      0.00      0.00       124
              military       0.70      0.09      0.15       223
                 water       0.78      0.19      0.30       416
                  food       0.81      0.48      0.60       724
               shelter       0.81      0.30      0.44       579
              clothing       0.60      0.06      0.11        97
                 money       0.60      0.04      0.08       141
        missing_people       0.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
# Define performance metric for use in grid search scoring object
def perform_metric(Y_T, Y_P):
    """Median F1 score for all classifiers
    
    inputs:
    Y_T: array. Array containing ArrayL labels.
    Y_P: array. Array containing ArrayP labels.
        
    Routputs:
    Median F1 score for all  classifiers
    """
    f1_list = []
    for i in range(np.shape(Y_P)[1]):
        f1 = f1_score(np.array(Y_T)[:, i], Y_P[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [14]:
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

scorer = make_scorer(perform_metric)
cv = GridSearchCV(pipeline, param_grid = parameters, scoring='f1_micro', n_jobs=-1, verbose = 10)


In [15]:
np.random.seed(74)
Best_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.6071716478070045, total=  54.6s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.6061100534856314, total=  54.6s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.6007097145976767, total=  56.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


...

[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  9.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.6042468118619938, total=  56.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 10.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.6270286133528981, total=  45.6s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.6301715291013048, total=  45.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.6287432209384578, total=  45.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.6439971035481534, total= 1.8min
[CV] clf__estimator__min_samples_split=2, clf__

[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed: 101.4min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
# evaluating metrics for test set
pred_test = Best_model.predict(X_test)

print(classification_report(Y_test.values,pred_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.93      0.89      4967
               request       0.79      0.53      0.63      1120
                 offer       0.00      0.00      0.00        23
           aid_related       0.71      0.71      0.71      2679
          medical_help       0.53      0.14      0.22       519
      medical_products       0.75      0.18      0.29       321
     search_and_rescue       0.58      0.05      0.08       153
              security       0.00      0.00      0.00       124
              military       0.57      0.09      0.16       223
                 water       0.87      0.44      0.59       416
                  food       0.82      0.66      0.73       724
               shelter       0.78      0.44      0.57       579
              clothing       0.80      0.25      0.38        97
                 money       0.69      0.06      0.12       141
        missing_people       0.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier()))
    ])

       
    
pipeline2.fit(X_train, Y_train)
y_prediction_test2 = pipeline2.predict(X_test)
print(classification_report(Y_test.values,y_prediction_test2, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.79      0.97      0.87      4967
               request       0.75      0.51      0.61      1120
                 offer       0.00      0.00      0.00        23
           aid_related       0.75      0.60      0.67      2679
          medical_help       0.58      0.22      0.32       519
      medical_products       0.64      0.33      0.43       321
     search_and_rescue       0.50      0.14      0.22       153
              security       0.25      0.03      0.06       124
              military       0.61      0.37      0.46       223
                 water       0.72      0.64      0.68       416
                  food       0.81      0.65      0.72       724
               shelter       0.74      0.51      0.60       579
              clothing       0.67      0.47      0.55        97
                 money       0.55      0.32      0.40       141
        missing_people       0.29      

### 9. Export your model as a pickle file

In [3]:
m = pickle.dumps('machinelearning.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.